In [ ]:
# import packages
import h5py
import numpy as np
import torch
from mirtorch.linear import NuSense, NuSenseGram, Diff3dgram, Diag
from mirtorch.alg.cg import CG
import os
import sys
from recutl import mri_coil_compress, resize_nd
from vis3d import tim

In [ ]:
# set variables
gpu_idx = 2 # GPU index for CUDA (-1 = CPU)
fname_kdata = '/home/djfrey/data/lps_fmri_20250411/lps_test/lps.h5' # name of the GRE data file
fname_smaps = os.path.join(os.path.dirname(fname_kdata), '../smaps.h5') # name of the smaps file
ncoil_comp = 16 # number of virtual coils to compress to
cutoff = 0.85 # kspace cutoff for echo-in/out filtering
rolloff = 0.2 # kspace rolloff for echo-in/out filtering
lam = 0.15 # regularization parameter for quadratic differencing penalty
niter = 50 # number of iterations for CG
M = None # reconstructed matrix size (None = N*cutoff)
ints2use = None # interleaf indices to use (None = all)
prjs2use = None # projection indices to use (None = all)
reps2use = 1 # repetition indices to use (None = all)
volwidth = None # number of projections per volume (None = use each rep as a volume)

In [ ]:
# select device
if torch.cuda.is_available() & (gpu_idx >= 0):
    device0 = torch.device(f'cuda:{gpu_idx}')
else:
    device0 = torch.device('cpu')
print(f'using device: {device0}')

using device: cuda:2


In [ ]:
# load in the data
with h5py.File(fname_kdata, 'r') as h5_file:
    kdata = h5_file['kdata/real'][:] + 1j * h5_file['kdata/imag'][:] # kspace data
    k_in = h5_file['ktraj/spoke_in'][:] # kspace spoke-in trajectory (1/cm)
    k_out = h5_file['ktraj/spoke_out'][:] # kspace spoke-out trajectory (1/cm)
    fov = h5_file['seq_args/fov'][0][0] # field of view (cm)
    N = int(h5_file['seq_args/N'][0][0]) # 3D matrix size
    nseg = int(h5_file['seq_args/nseg'][0][0]) # number of points per segment
    nspokes = int(h5_file['seq_args/nspokes'][0][0]) # number of spokes
    nprj = int(h5_file['seq_args/nprj'][0][0]) # number of projections
    nint = int(h5_file['seq_args/nint'][0][0]) # number of interleaves
    nrep = int(h5_file['seq_args/nrep'][0][0]) # number of repetitions
    ncoil = int(h5_file['ncoil'][0][0]) # number of coils

# convert to tensors
kdata = torch.tensor(kdata).reshape(ncoil,nrep,nprj,nint,nseg*nspokes)
k_in = torch.tensor(k_in).reshape(3,nrep,nprj,nint,nseg*nspokes)
k_out = torch.tensor(k_out).reshape(3,nrep,nprj,nint,nseg*nspokes)

In [ ]:
# set default values for interleaves, projections, and repetitions to use
if ints2use is None:
    ints2use = nint
if prjs2use is None:
    prjs2use = nprj
if reps2use is None:
    reps2use = nrep
if volwidth is None:
    volwidth = prjs2use*ints2use

# get number of volumes
nvol = reps2use*prjs2use*ints2use // volwidth

# arrange data into volumes
kdata2 = kdata.clone()
kdata2 = kdata2[:,np.arange(reps2use),:,:,:]
kdata2 = kdata2[:,:,np.arange(prjs2use),:,:]
kdata2 = kdata2[:,:,:,np.arange(ints2use),:]
kdata2 = kdata2.reshape(ncoil,reps2use*prjs2use*ints2use,nseg*nspokes)
kdata2 = kdata2[:,:nvol*volwidth,:]
kdata2 = kdata2.reshape(ncoil,nvol,volwidth*nseg*nspokes)
kdata2 = kdata2.permute(1,0,2)

# arrange kspace trajectory into volumes
k_in2 = k_in.clone()
k_in2 = k_in2[:,np.arange(reps2use),:,:,:]
k_in2 = k_in2[:,:,np.arange(prjs2use),:,:]
k_in2 = k_in2[:,:,:,np.arange(ints2use),:]
k_in2 = k_in2.reshape(3,reps2use*prjs2use*ints2use,nseg*nspokes)
k_in2 = k_in2[:,:nvol*volwidth,:]
k_in2 = k_in2.reshape(3,nvol,volwidth*nseg*nspokes)
k_in2 = k_in2.permute(1,0,2)

k_out2 = k_out.clone()
k_out2 = k_out2[:,np.arange(reps2use),:,:,:]
k_out2 = k_out2[:,:,np.arange(prjs2use),:,:]
k_out2 = k_out2[:,:,:,np.arange(ints2use),:]
k_out2 = k_out2.reshape(3,reps2use*prjs2use*ints2use,nseg*nspokes)
k_out2 = k_out2[:,:nvol*volwidth,:]
k_out2 = k_out2.reshape(3,nvol,volwidth*nseg*nspokes)
k_out2 = k_out2.permute(1,0,2)

# calculate reconstructed matrix size
if M is None:
    M = int(np.ceil(N*cutoff))

In [ ]:
# load in the sensitivity maps
with h5py.File(fname_smaps, 'r') as h5_file:
    smaps = torch.tensor(h5_file['/real'][:] + 1j * h5_file['/imag'][:]).unsqueeze(0).to(kdata) # kspace data
smaps = resize_nd(smaps, (2,3,4), M/smaps.shape[2]) # resize to match kspace data
smaps = smaps.permute(0,1,4,3,2)

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [ ]:
# coil compress the data
kdata_comp,Vr = mri_coil_compress(kdata2, ncoil=ncoil_comp)

# coil compress the sensitivity maps
smaps_comp,_ = mri_coil_compress(smaps, Vr=Vr)

In [ ]:
# convert trajectory to spatial frequencies
om_in = 2*torch.pi * fov/M * k_in2
om_out = 2*torch.pi * fov/M * k_out2

# create filter objects
r_cut = cutoff * N/M * torch.pi
r_roll = rolloff * N/M * torch.pi
kfilt = lambda r: (r <= r_cut) * 1/(1 + torch.exp(2*torch.pi * (r - r_cut)/r_roll))
Hvec_in = kfilt(torch.norm(om_in,2,dim=1,keepdim=True)).repeat(1,ncoil_comp,1)
Hvec_out = kfilt(torch.norm(om_out,2,dim=1,keepdim=True)).repeat(1,ncoil_comp,1)
H_in = Diag(Hvec_in.to(device0))
H_out = Diag(Hvec_out.to(device0))

# create nufft system operators with flat sensitivity
FS_in = NuSense(smaps_comp.to(device0), om_in.to(device0), nbatch=nvol)
FS_out = NuSense(smaps_comp.to(device0), om_out.to(device0), nbatch=nvol)

# set up system matrices and data
A = H_in*FS_in + H_out*FS_out
G_in = NuSenseGram(smaps_comp.to(device0), om_in.to(device0), kweights=Hvec_in.to(device0), nbatch=nvol)
G_out = NuSenseGram(smaps_comp.to(device0), om_out.to(device0), kweights=Hvec_out.to(device0), nbatch=nvol)
AHA = G_in + G_out + 2*(H_in*FS_in).H*(H_out*FS_out)

# add L2 roughness penalty
THT = Diff3dgram(FS_in.size_in)
AHA_tikh = AHA + lam*THT

# set up data
y = kdata_comp.to(device0)
AHy = A.H * y

# set up the CG solver
solv = CG(AHA_tikh, max_iter=niter)

In [ ]:
# solve with CG
x = solv.run(torch.zeros(nvol,1,M,M,M).to(kdata).to(device0), AHy)

In [ ]:
# visualize the results
from vis3d import tim
tim(x)
tim(x,viewtype='mid3')

UnboundLocalError: cannot access local variable 'Nz' where it is not associated with a value